In [9]:
import requests


In [11]:
def request_gpt(user_text, temperature=0.7, top_p=0.95, max_tokens=800):

    # endpoint
    # method
    # header
    # payload

    api_base="https://team10-eighti.openai.azure.com" 
    deployment_id="gpt-4o"  
    endpoint = f"{api_base}/openai/deployments/{deployment_id}/chat/completions?api-version=2024-08-01-preview"
    api_key = '1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw'
    
    search_endpoint="https://team10-eighti-search.search.windows.net"
    search_key = 'wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3'
    search_index="sign-index"
    semantic_name = "sign-semantic"

    method = requests.post
    headers = {
        "Content-Type": "application.json",
        "api-key": api_key
    }
    payload = {
        "messages": [
            {
                "role": "system",
                "content": "당신은 사용자가 수화 정보를 찾는데 도움을 주는 수화 AI 도우미입니다."
            },
            {
                "role": "user",
                "content": user_text
            }
        ],
        "temperature": temperature,
        "top_p": top_p,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "max_tokens": max_tokens,
        "stop": None,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": search_index,
                    "semantic_configuration": semantic_name,
                    "query_type": "semantic",
                    "fields_mapping": {},
                    "filter": None,
                    "top_n_documents": 5,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    }
                }
            }
        ]
    }

    response = requests.post(endpoint, headers=headers, json=payload)
    # print(response.status_code)
    # print(response.text)

    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json['choices'][0]['message']['content']
        citations = response_json['choices'][0]['message']['context']['citations'][0]['content'].split('\n')
        return content_text, citations
    else:
        return ""

content_text, citations_list = request_gpt("된장찌개가 수화로 어떻게 돼?", temperature=0.7, top_p=0.95, max_tokens=800)

print(content_text)
print(citations_list)


된장찌개를 수화로 표현할 때는 오른손의 손가락을 왼손의 5지와 1·2·3·4지 사이에 넣고 오른손 등이 스치게 한 다음, 손바닥이 위로 향하게 하여 약간 굽힌 왼손 밑에서 같은 모양의 오른손을 두 번 오르내립니다 [doc1].
['음식', '손끝이 아래로 향하게 구부린 오른손의 손가락을 왼손의 5지와 1·2·3·4지 사이에 넣고 오른손 등이 스치게 뺀 다음, 손바닥이 위로 향하게 펴서 약간 구부린 왼손 밑에서 같은 모양의 오른손을 두 번 오르내린다.', 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201224/791296/IMG000357950_700X466.jpg http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151126/217119/IMG000208379_700X466.jpg', 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201210/784487/MOV000356911_700X466.mp4', '된장찌개']


In [ ]:
import gradio as gr
import os
import cv2
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import io
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"
 
# Initialize the prediction client
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

def draw_boxes(image, predictions):
    """Draw bounding boxes on the image based on predictions"""
    img = image.copy()
    for pred in predictions:
        if pred.probability > 0.9 :
            color = (255, 0, 0)
            box = pred.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])
        
        # Draw rectangle
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
        
        # Add label with confidence score
            label = f"{pred.tag_name}: {pred.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img

def process_frame(frame):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame = draw_boxes(frame, results.predictions)
        
        return annotated_frame
    
    except Exception as e:
        print(f"Error during prediction: {e}")
        return frame


# Checklist section
with gr.Blocks() as demo:  
            
    interface = gr.Interface(
        fn=process_frame,
        inputs=gr.Image(sources="webcam", streaming=True, mirror_webcam=True),
        outputs=gr.Image(label="Detected Objects"),
        live=True,
        title="Azure Custom Vision Object Detection",
        description="Real-time object detection using Azure Custom Vision",
        examples=[],
        cache_examples=False
    )

if __name__ == "__main__":
    demo.launch(share=True)

In [13]:
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content  # 속성 접근 방식으로 수정
        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history

    except Exception as e:
        return [("Error", str(e))], chat_history

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# Azure OpenAI + Cognitive Search + Speech 기반 수화 챗봇")
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
    clear_button = gr.Button("Clear Chat")
    
    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, state])
    clear_button.click(lambda: [], None, chatbot)

# 실행
demo.launch()

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [14]:
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content
        chat_history.append((user_input, assistant_reply))  # 대화 기록 저장

        # Azure Speech Service로 응답 읽기
        speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, assistant_reply

    except Exception as e:
        return [("Error", str(e))], chat_history

# 참고 자료 검색 및 로드 함수
def fetch_references(user_input):
    try:
        # 검색된 사진과 동영상 URL (샘플 데이터베이스나 API 연동 필요)
        references = {
            "images": [
                "https://example.com/image1.jpg",
                "https://example.com/image2.jpg"
            ],
            "videos": [
                "https://example.com/video1.mp4",
                "https://example.com/video2.mp4"
            ]
        }
        return references["images"], references["videos"]
    except Exception as e:
        return [], []

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# Azure OpenAI + Cognitive Search + Speech 기반 수화 학습 챗봇")
    
    # 채팅창 구성
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
    clear_button = gr.Button("Clear Chat")
    references_button = gr.Button("참고 자료 보기")

    # 참고 자료 (이미지 및 동영상)
    images_output = gr.Gallery(label="참고 이미지").style(grid=2)
    videos_output = gr.Video(label="참고 영상")
    
    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, state])
    clear_button.click(lambda: [], None, chatbot)
    references_button.click(fetch_references, [user_input], [images_output, videos_output])

# 실행
if __name__ == "__main__":
    demo.launch()


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


AttributeError: 'Gallery' object has no attribute 'style'

In [ ]:
import os
import pandas as pd
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer

# CSV 파일 경로
csv_file_path = "이거 사용.CSV"

# CSV 파일 읽기
data = pd.read_csv(csv_file_path, encoding="utf-8")

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 단어 리스트 추출
word_list = data["단어명"].tolist()

# 단어에 따른 동영상과 이미지 불러오기 함수
def fetch_media(selected_word):
    # 선택된 단어에 해당하는 행 필터링
    row = data[data["단어명"] == selected_word]
    
    if not row.empty:
        video_url = row["동영상"].values[0]
        image_url = row["이미지"].values[0]
        description = row["설명"].values[0]
        return video_url, image_url, description
    else:
        return None, None, "해당 단어를 찾을 수 없습니다."

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content
        chat_history.append((user_input, assistant_reply))  # 대화 기록 저장

        # Azure Speech Service로 응답 읽기
        speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history

    except Exception as e:
        return [("Error", str(e))], chat_history

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# Azure OpenAI + 수어 학습 프로그램")

    with gr.Row():
        # 왼쪽: 채팅 및 음성 기능
        with gr.Column():
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            
            # 대화 기록 저장
            state = gr.State([])

            # 이벤트 연결
            user_input.submit(chat_with_openai, [user_input, state], [chatbot, state])
            clear_button.click(lambda: [], None, chatbot)

        # 오른쪽: 단어 선택, 동영상, 이미지 출력
        with gr.Column():
            selected_word = gr.Dropdown(label="단어를 선택하세요", choices=word_list)
            video_output = gr.Video(label="동영상")
            image_output = gr.Image(label="이미지")
            description_output = gr.Textbox(label="설명", interactive=False)
            
            # 이벤트 연결
            selected_word.change(fetch_media, inputs=[selected_word], outputs=[video_output, image_output, description_output])

# 실행
if __name__ == "__main__":
    demo.launch()


In [ ]:
import os
import pandas as pd
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer

# CSV 파일 경로
csv_file_path = "이거 사용.CSV"

# CSV 파일 읽기
data = pd.read_csv(csv_file_path, encoding="utf-8")

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 채팅 및 미디어 출력 함수
def chat_with_openai_and_fetch_media(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
        )

        # 응답 추출
        assistant_reply = completion.choices[0].message.content

        # Azure Speech Service로 응답 읽기
        speech_synthesizer.speak_text_async(assistant_reply)

        # CSV 파일에서 단어에 해당하는 동영상 및 이미지 찾기
        row = data[data["단어명"] == user_input.strip()]
        if not row.empty:
            video_url = row["동영상"].values[0]
            image_url = row["이미지"].values[0]
        else:
            video_url = None
            image_url = None

        # 대화 기록 업데이트
        chat_history.append((user_input, assistant_reply))

        # 반환: 대화 기록, 동영상 URL, 이미지 URL
        return chat_history, video_url, image_url

    except Exception as e:
        return [("Error", str(e))], None, None

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# Azure OpenAI + 수어 학습 프로그램")

    with gr.Row():
        # 왼쪽: 채팅 및 음성 기능
        with gr.Column():
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            
            # 대화 기록 저장
            state = gr.State([])

            # 이벤트 연결
            user_input.submit(chat_with_openai_and_fetch_media, [user_input, state], [chatbot, state])
            clear_button.click(lambda: [], None, chatbot)

        # 오른쪽: 동영상, 이미지 출력
        with gr.Column():
            video_output = gr.Video(label="동영상")
            image_output = gr.Image(label="이미지")
            
            # 채팅 입력에 따라 동영상 및 이미지 업데이트
            user_input.submit(chat_with_openai_and_fetch_media, [user_input, state], [chatbot, video_output, image_output])

# 실행
if __name__ == "__main__":
    demo.launch()


In [5]:
import os
import pandas as pd
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer

# CSV 파일 경로
csv_file_path = "이거 사용.CSV"

# CSV 파일 읽기
data = pd.read_csv(csv_file_path, encoding="utf-8")

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 단어 리스트 추출
word_list = data["단어명"].tolist()

# 단어에 따른 동영상과 이미지 불러오기 함수
def fetch_media(selected_word):
    # 선택된 단어에 해당하는 행 필터링
    row = data[data["단어명"] == selected_word]
    
    if not row.empty:
        video_url = row["동영상"].values[0]
        image_url = row["이미지"].values[0]
        description = row["설명"].values[0]
        return video_url, image_url, description
    else:
        return None, None, "해당 단어를 찾을 수 없습니다."

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content
        chat_history.append((user_input, assistant_reply))  # 대화 기록 저장

        # Azure Speech Service로 응답 읽기
        speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history

    except Exception as e:
        return [("Error", str(e))], chat_history

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# Azure OpenAI + 수어 학습 프로그램")

    with gr.Row():
        # 왼쪽: 채팅 및 음성 기능
        with gr.Column():
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
            
            # 대화 기록 저장
            state = gr.State([])

            # 이벤트 연결
            user_input.submit(chat_with_openai, [user_input, state], [chatbot, state])
            clear_button.click(lambda: [], None, chatbot)

        # 오른쪽: 단어 선택, 동영상, 이미지 출력
        with gr.Column():
            video_output = gr.Video(label="동영상")
            image_output = gr.Image(label="이미지")
            
            # 채팅 입력에 따라 동영상 및 이미지 업데이트
            user_input.submit(chat_with_openai_and_fetch_media, [user_input, state], [chatbot, video_output, image_output])


# 실행
if __name__ == "__main__":
    demo.launch()


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
